In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#%cd /content/drive/MyDrive

/content/drive/MyDrive


In [68]:
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

# define root directory 
ROOT = os.getcwd()
print(ROOT)

dataset_dir = os.path.join(ROOT,"Vision")
scenario_dir = os.path.join(dataset_dir,"Scenario6")
development_dir = os.path.join(scenario_dir, "development_dataset")
challenge_dir = os.path.join(scenario_dir, "challenge_dataset")

f:\pblproject


In [69]:
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available() :
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [81]:
# load data

import torchvision
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms.functional as FT

def transform(image, flag) :
    mean = [0.485, 0.456, 0.406]
    std =  [0.229, 0.224, 0.225]

    if flag == 2 :      #test
        new_image = FT.resize(image,224)
        new_image = FT.to_tensor(new_image)
        new_image = FT.normalize(new_image, mean=mean, std=std)
    else :              #train, val
        new_image = FT.resize(image,224)
        new_image = FT.to_tensor(new_image)
        new_image = FT.normalize(new_image, mean=mean, std=std)

    return new_image

class DeepSense6G(Dataset):
    def __init__(self, data_folder, split, flag):
        self.split = split
        assert self.split in {'Scenario5','Scenario6','Scenario7','Scenario8','Scenario9'}

        self.data_folder = data_folder
        self.scenario_path = os.path.join(self.data_folder, split)
        self.development_path = os.path.join(self.scenario_path, 'development_dataset')
        self.challenge_path = os.path.join(self.scenario_path, 'challenge_dataset')

        self.flag = flag

        self.split = self.split.lower()
        if flag == 0 :      #train
            self.dataframe_path = os.path.join(self.development_path, self.split + '_dev_train.csv')
        elif flag == 1 :    #validation
            self.dataframe_path = os.path.join(self.development_path, self.split + '_dev_val.csv')
        elif flag == 2 :              #test
            self.dataframe_path = os.path.join(self.development_path, self.split + '_dev_test.csv')


        self.dataframe = pd.read_csv(self.dataframe_path)

        self.Images_path = self.dataframe['unit1_rgb_1'].values
        self.Beam_index = self.dataframe['beam_index_1'].values
        self.image_index = self.dataframe['index'].values
        self.Beam_pwr_path = self.dataframe['unit1_pwr_1'].values

    def __getitem__(self, i):

        self.img_rel_path = self.Images_path[i]
        if self.flag == 2 :     #test
            self.img_abs_path = os.path.join(self.development_path, self.img_rel_path)
        else :                  #train, validation
            self.img_abs_path = os.path.join(self.development_path, self.img_rel_path)
        
        image = Image.open(self.img_abs_path, mode='r')
        image = image.convert('RGB')

        beam_index = self.Beam_index[i]

        image = transform(image, self.flag)

        return image, beam_index

    def __len__(self):
        return len(self.Images_path)

    # def _collate_fn(self, batch_size) :

    #     images = list()
    #     beam_indexes = list()

    #     for b in batch_size :
    #         images.append(b[0])
    #         beam_indexes.append(b[1])

    #     images = torch.stack(images, dim=0)

    #     return images, beam_indexes

In [71]:
# device initialize

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(DEVICE)

cuda


In [72]:
# pre-trained model(ResNet50)

from torchvision import models

resnet50 = models.resnet50(pretrained=True)
num_ftrs = resnet50.fc.in_features
resnet50.fc = torch.nn.Linear(num_ftrs, 64)

model = resnet50.to(DEVICE)

e:\anaconda3\envs\test_envs\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
e:\anaconda3\envs\test_envs\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [73]:
# #%pip install torchsummary  #model summary
# from torchsummary import summary 

# def summary_model(model, input_shape=(3,224,224)) :
#     model = model.cuda()
#     summary(model, input_shape)

# print(model)
# summary_model(model)

In [74]:
#training
import time

batch_size = 32
scenario = 'Scenario6'

train_dataset = DeepSense6G(dataset_dir, split=scenario, flag=0)
val_dataset = DeepSense6G(dataset_dir, split=scenario, flag=1)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False) 

epochs = 25
best_loss = 100

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss = torch.nn.CrossEntropyLoss().cuda()

train_acc_list = []
val_acc_list = []

for epoch in range(epochs) :
    start = time.time()
    print(f'Epoch {epoch}/{epochs}')

    #train
    train_loss = 0
    correct = 0

    model.train()
    for img, bi in tqdm(train_loader) :

        optimizer.zero_grad()
        y_pred = model(img.cuda())
        cost = loss(y_pred, bi.cuda())

        cost.backward()
        optimizer.step()
        train_loss += cost.item()

        pred = y_pred.data.max(1, keepdim=True)[1]
        correct += pred.cpu().eq(bi.data.view_as(pred)).sum()

    train_loss /= len(train_loader)
    train_acc = correct / len(train_loader.dataset)
    train_acc_list.append(train_acc)

    #validate
    val_loss = 0
    val_correct = 0

    with torch.no_grad() :
        model.eval()
        for img, bi in val_loader :
            y_pred = model(img.cuda())
            cost = loss(y_pred, bi.cuda())
            val_loss += cost.item()

            pred = y_pred.data.max(1, keepdim=True)[1]
            val_correct += pred.cpu().eq(bi.data.view_as(pred)).cpu().sum()
        
        val_loss /= len(val_loader)
        val_acc = val_correct / len(val_loader.dataset)
        val_acc_list.append(val_acc)

        if val_loss < best_loss :
            torch.save({
                'epoch' : epoch,
                'model' : model,
                'model_state_dict' : model.state_dict(),
                'optimizer_state_dict' : optimizer.state_dict(),
                'loss' : cost.item,
            }, os.path.join(dataset_dir, scenario + '/checkpoint/bestCheckpoint.pth'))

            print(f'Epoch {epoch:05d}: val_loss improved from {best_loss:.5f} to {val_loss:.5f}, saving model to bestCheckPoint.pth')
            best_loss = val_loss
        else :
            print(f'Epoch {epoch:05d}: val_loss did not improve')
    print(f'{int(time.time() - start)}s - loss: {train_loss: .5f} - acc: {train_acc:.5f} - val_loss: {val_loss:.5f} - val_acc: {val_acc:.5f}')

Epoch 0/25


100%|██████████| 21/21 [00:16<00:00,  1.27it/s]


Epoch 00000: val_loss improved from 100.00000 to 3.09195, saving model to bestCheckPoint.pth
19s - loss:  3.86420 - acc: 0.12251 - val_loss: 3.09195 - val_acc: 0.30612
Epoch 1/25


100%|██████████| 21/21 [00:13<00:00,  1.60it/s]


Epoch 00001: val_loss improved from 3.09195 to 2.50618, saving model to bestCheckPoint.pth
16s - loss:  2.84155 - acc: 0.34916 - val_loss: 2.50618 - val_acc: 0.42857
Epoch 2/25


...

Epoch 00022: val_loss did not improve
16s - loss:  0.09831 - acc: 0.98469 - val_loss: 2.38026 - val_acc: 0.42857
Epoch 23/25


100%|██████████| 21/21 [00:13<00:00,  1.53it/s]


Epoch 00023: val_loss did not improve
16s - loss:  0.07620 - acc: 0.98775 - val_loss: 2.34513 - val_acc: 0.44898
Epoch 24/25


100%|██████████| 21/21 [00:13<00:00,  1.54it/s]


Epoch 00024: val_loss did not improve
16s - loss:  0.09066 - acc: 0.98469 - val_loss: 2.54303 - val_acc: 0.44898


In [75]:
## 저장되어있는 best checkpoint load

best_model = torch.load(os.path.join(dataset_dir, scenario + '/checkpoint/bestCheckpoint.pth'))['model'] 
best_model.load_state_dict(torch.load(os.path.join(dataset_dir, scenario + '/checkpoint/bestCheckpoint.pth'))['model_state_dict'])  

<All keys matched successfully>

In [96]:
#test 
test_dataset = DeepSense6G(dataset_dir, split=scenario, flag=2)
#print(data)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

correct = 0
total = 0

with torch.no_grad() :
    model.eval()
    for img, bi in test_loader :
        pred = model(img.cuda())
        _, predicted = torch.max(pred.data, 1)
        total += bi.size(0)
        #correct += (predicted == bi.cudas()).sum().item()
        correct += (predicted == bi.cuda()).sum().item()

print('accuracy:', (100 * correct / total),"%")

accuracy: 50.0 %
